<a href="https://colab.research.google.com/github/joony0512/Linear_Programming/blob/main/Sales_cost_maximize2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 57.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 불러오기
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


Data =pd.read_excel('/content/drive/MyDrive/2023년 4학년 2학기/OA/기말고사/분석 데이터.xlsx')


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint

#! Income 데이터를 수익별로 그룹화
bins=[0,10001,20001,30001,40001,50001,60001,70001,80001,90001,100001]
Data['Income_M_cost'] = np.digitize(Data['Income'], bins) * 100.0


#! Education과 Marital_Status를 매핑해서 그룹별 마케팅비용 추가
education_mapping = {
    'Basic': 20,
    '2n Cycle': 10,
    'Graduation': 30,
    'Master': 40,
    'PhD': 50
}
Data['Education_M_cost'] = Data['Education'].map(education_mapping).astype(int)

marital_status_mapping = {
    'Single': 30,
    'Married':50,
    'Divorced': 10,
    'Together': 10,
    'Widow': 10,
    'Alone': 10,
    'Absurd':10,
    'YOLO' : 10    
}
Data['Marital_M_cost'] = Data['Marital_Status'].map(marital_status_mapping).astype(int)

#! 현재 데이터 기준 그룹별 예측비용 Column 추가
Data['profit'] = Data['MntPurchases'] - (Data['Income_M_cost']+Data['Education_M_cost']+Data['Marital_M_cost'])

#? 결혼여부, 교육정보 변수를 숫자로 인코딩.
label_encoder = LabelEncoder()
Data['Marital_Status'] = label_encoder.fit_transform(Data['Marital_Status']) 
Data['Education'] = label_encoder.fit_transform(Data['Education'])  



In [30]:
Data

,Education,Marital_Status,Income,MntPurchases,Income_M_cost,Education_M_cost,Marital_M_cost,profit
0,0,2,87305,1152,900.0,10,10,232.0
1,2,5,42011,189,500.0,30,10,-351.0
2,2,3,28442,57,300.0,30,50,-323.0
3,4,2,70179,1034,800.0,50,10,174.0
4,4,5,76140,1348,800.0,50,10,488.0
...,...,...,...,...,...,...,...,...
1795,2,5,50127,320,600.0,30,10,-320.0
1796,2,3,90765,1724,1000.0,30,50,644.0
1797,2,4,42160,140,500.0,30,30,-420.0
1798,4,3,67353,913,700.0,50,50,113.0


In [6]:
features = ['Education_M_cost', 'Marital_M_cost', 'Income_M_cost']
target = 'MntPurchases'

grouped_data = Data.groupby(features).mean().reset_index()
# grouped_data

In [7]:
# # 특성(독립 변수)와 타깃(종속) 분리
X = grouped_data[features]
y = grouped_data[target]

#  훈련 세트와 테스트 세트로 나누기.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)

# Random Forest 모델 생성 및 학습
model = RandomForestRegressor(max_depth= 10, min_samples_leaf= 3, min_samples_split= 2, n_estimators= 10, random_state= 35)
model.fit(X_train, y_train)

# 예측 및 성능평가
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

predicted_Profit = model.predict(X)

# 예측된 MntPurchases 정보 추가
grouped_data['PredictedMnt'] = predicted_Profit


# 최종 결과 확인
grouped_data

,Education_M_cost,Marital_M_cost,Income_M_cost,Education,Marital_Status,Income,MntPurchases,profit,PredictedMnt
0,10,10,100.0,0.0,5.000000,7500.000000,76.500000,-43.500000,114.106992
1,10,10,200.0,0.0,4.500000,15795.333333,70.833333,-149.166667,57.041476
2,10,10,300.0,0.0,4.846154,25475.461538,82.923077,-237.076923,70.065049
3,10,10,400.0,0.0,3.500000,34393.000000,93.125000,-326.875000,153.612001
4,10,10,500.0,0.0,4.111111,45273.555556,505.555556,-14.444444,292.678858
...,...,...,...,...,...,...,...,...,...
116,50,50,600.0,4.0,3.000000,55065.843750,534.937500,-165.062500,540.965515
117,50,50,700.0,4.0,3.000000,65596.931034,1128.137931,328.137931,978.353371
118,50,50,800.0,4.0,3.000000,75103.894737,1368.263158,468.263158,1318.385724
119,50,50,900.0,4.0,3.000000,83829.642857,1492.642857,492.642857,1489.807562


In [9]:
print('오차:', mse)


오차: 4046.647150674442


In [19]:
result = grouped_data[['Education_M_cost','Marital_M_cost','Income_M_cost','MntPurchases','PredictedMnt','profit']]
result

,Education_M_cost,Marital_M_cost,Income_M_cost,MntPurchases,PredictedMnt,profit
0,10,10,100.0,76.500000,114.106992,-43.500000
1,10,10,200.0,70.833333,57.041476,-149.166667
2,10,10,300.0,82.923077,70.065049,-237.076923
3,10,10,400.0,93.125000,153.612001,-326.875000
4,10,10,500.0,505.555556,292.678858,-14.444444
...,...,...,...,...,...,...
116,50,50,600.0,534.937500,540.965515,-165.062500
117,50,50,700.0,1128.137931,978.353371,328.137931
118,50,50,800.0,1368.263158,1318.385724,468.263158
119,50,50,900.0,1492.642857,1489.807562,492.642857


In [20]:
result['Marketing_cost'] =result['Education_M_cost']+result['Marital_M_cost']+result['Income_M_cost']

<ipython-input-20-db5db4236867>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Marketing_cost'] =result['Education_M_cost']+result['Marital_M_cost']+result['Income_M_cost']


In [32]:
result['Total_Profit']= result['PredictedMnt'] - (result['Education_M_cost']+result['Marital_M_cost']+result['Income_M_cost'])

<ipython-input-32-fc32dd6cbe40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Total_Profit']= result['PredictedMnt'] - (result['Education_M_cost']+result['Marital_M_cost']+result['Income_M_cost'])


In [33]:
result

,Education_M_cost,Marital_M_cost,Income_M_cost,MntPurchases,PredictedMnt,profit,Marketing_cost,Total_Profit
0,10,10,100.0,76.500000,114.106992,-43.500000,120.0,-5.893008
1,10,10,200.0,70.833333,57.041476,-149.166667,220.0,-162.958524
2,10,10,300.0,82.923077,70.065049,-237.076923,320.0,-249.934951
3,10,10,400.0,93.125000,153.612001,-326.875000,420.0,-266.387999
4,10,10,500.0,505.555556,292.678858,-14.444444,520.0,-227.321142
...,...,...,...,...,...,...,...,...
116,50,50,600.0,534.937500,540.965515,-165.062500,700.0,-159.034485
117,50,50,700.0,1128.137931,978.353371,328.137931,800.0,178.353371
118,50,50,800.0,1368.263158,1318.385724,468.263158,900.0,418.385724
119,50,50,900.0,1492.642857,1489.807562,492.642857,1000.0,489.807562


In [39]:
import pandas as pd
from pulp import *

# 데이터 불러오기
data = result[['Marketing_cost', 'Total_Profit']] 

# 변수 선택
features = 'Marketing_cost'
target = 'Total_Profit'

# LPproblem 설정
problem = LpProblem("VIP_Selection", LpMaximize)

# 변수 생성
variables = []
for index, row in data.iterrows():
    variable = LpVariable(f"Customer_{index}", lowBound=0, upBound=1, cat='Binary')
    variables.append(variable)

# 목적 함수 설정
problem += lpSum(variables[i] * data.loc[i, target] for i in range(len(variables)))

# 제약 조건 설정
problem += lpSum(variables[i] * data.loc[i, features] for i in range(len(variables))) <= 5000

# 문제 풀이
problem.solve()

# 결과 출력
print("Variable Values:")
for variable in variables:
    print(f"{variable.name}: {variable.value()}")

print("VIP List:")
vip_list = []
for i, variable in enumerate(variables):
    if variable.varValue > 0:
        vip_list.append((i, data.loc[i, features]))
        print(f"Row: {i}, Features: {data.loc[i, features]}")

Total_cost = sum(vip[1] for vip in vip_list)
print("Total Cost:", Total_cost)


Variable Values:
Customer_0: 0.0
Customer_1: 0.0
Customer_2: 0.0
Customer_3: 0.0
Customer_4: 0.0
Customer_5: 0.0
Customer_6: 0.0
Customer_7: 0.0
Customer_8: 0.0
Customer_9: 0.0
Customer_10: 0.0
Customer_11: 0.0
Customer_12: 0.0
Customer_13: 0.0
Customer_14: 0.0
Customer_15: 0.0
Customer_16: 0.0
Customer_17: 0.0
Customer_18: 0.0
Customer_19: 0.0
Customer_20: 0.0
Customer_21: 0.0
Customer_22: 0.0
Customer_23: 0.0
Customer_24: 0.0
Customer_25: 0.0
Customer_26: 0.0
Customer_27: 0.0
Customer_28: 0.0
Customer_29: 0.0
Customer_30: 0.0
Customer_31: 0.0
Customer_32: 0.0
Customer_33: 0.0
Customer_34: 0.0
Customer_35: 0.0
Customer_36: 0.0
Customer_37: 0.0
Customer_38: 0.0
Customer_39: 0.0
Customer_40: 0.0
Customer_41: 0.0
Customer_42: 0.0
Customer_43: 0.0
Customer_44: 1.0
Customer_45: 0.0
Customer_46: 0.0
Customer_47: 0.0
Customer_48: 0.0
Customer_49: 0.0
Customer_50: 0.0
Customer_51: 0.0
Customer_52: 0.0
Customer_53: 0.0
Customer_54: 1.0
Customer_55: 0.0
Customer_56: 0.0
Customer_57: 0.0
Custome

In [40]:
vip_num = [vip[0] for vip in vip_list]
result.loc[vip_num]

,Education_M_cost,Marital_M_cost,Income_M_cost,MntPurchases,PredictedMnt,profit,Marketing_cost,Total_Profit
44,30,10,1000.0,2181.666667,1762.987588,1141.666667,1040.0,722.987588
54,30,30,1000.0,1789.666667,1786.391421,729.666667,1060.0,726.391421
71,40,10,800.0,1386.384615,1366.711099,536.384615,850.0,516.711099
81,40,30,900.0,1653.000000,1529.237378,683.000000,970.0,559.237378
82,40,30,1000.0,1998.000000,1812.563485,928.000000,1070.0,742.563485


In [41]:
sum(result.loc[vip_num]['Total_Profit'])

3267.8909700789236

In [42]:
import pandas as pd
from pulp import *

# 데이터 불러오기
data = result[['Marketing_cost', 'Total_Profit']] 

# 변수 선택
features = 'Marketing_cost'
target = 'Total_Profit'

# LPproblem 설정
problem = LpProblem("VIP_Selection", LpMaximize)

# 변수 생성
variables = []
for index, row in data.iterrows():
    variable = LpVariable(f"Customer_{index}", lowBound=0, upBound=1, cat='Binary')
    variables.append(variable)

# 목적 함수 설정
problem += lpSum(variables[i] * data.loc[i, target] for i in range(len(variables)))

# 제약 조건 설정
problem += lpSum(variables[i] * data.loc[i, features] for i in range(len(variables))) <= 30000

# 문제 풀이
problem.solve()

# 결과 출력
print("Variable Values:")
for variable in variables:
    print(f"{variable.name}: {variable.value()}")

print("VIP List:")
vip_list = []
for i, variable in enumerate(variables):
    if variable.varValue > 0:
        vip_list.append((i, data.loc[i, features]))
        print(f"Row: {i}, Features: {data.loc[i, features]}")

Total_cost = sum(vip[1] for vip in vip_list)
print("Total Cost:", Total_cost)


Variable Values:
Customer_0: 0.0
Customer_1: 0.0
Customer_2: 0.0
Customer_3: 0.0
Customer_4: 0.0
Customer_5: 0.0
Customer_6: 0.0
Customer_7: 1.0
Customer_8: 1.0
Customer_9: 1.0
Customer_10: 0.0
Customer_11: 0.0
Customer_12: 0.0
Customer_13: 0.0
Customer_14: 0.0
Customer_15: 1.0
Customer_16: 1.0
Customer_17: 0.0
Customer_18: 0.0
Customer_19: 0.0
Customer_20: 0.0
Customer_21: 0.0
Customer_22: 0.0
Customer_23: 0.0
Customer_24: 0.0
Customer_25: 1.0
Customer_26: 0.0
Customer_27: 0.0
Customer_28: 0.0
Customer_29: 1.0
Customer_30: 0.0
Customer_31: 0.0
Customer_32: 0.0
Customer_33: 0.0
Customer_34: 0.0
Customer_35: 0.0
Customer_36: 0.0
Customer_37: 0.0
Customer_38: 0.0
Customer_39: 0.0
Customer_40: 0.0
Customer_41: 0.0
Customer_42: 1.0
Customer_43: 1.0
Customer_44: 1.0
Customer_45: 0.0
Customer_46: 0.0
Customer_47: 0.0
Customer_48: 0.0
Customer_49: 0.0
Customer_50: 0.0
Customer_51: 0.0
Customer_52: 1.0
Customer_53: 1.0
Customer_54: 1.0
Customer_55: 0.0
Customer_56: 0.0
Customer_57: 0.0
Custome

In [43]:
vip_num = [vip[0] for vip in vip_list]
result.loc[vip_num]

,Education_M_cost,Marital_M_cost,Income_M_cost,MntPurchases,PredictedMnt,profit,Marketing_cost,Total_Profit
7,10,10,800.0,920.200000,1241.092159,100.200000,820.0,421.092159
8,10,10,900.0,1267.000000,1403.150697,347.000000,920.0,483.150697
9,10,10,1000.0,1263.000000,1654.744992,243.000000,1020.0,634.744992
15,10,30,800.0,1307.500000,1241.092159,467.500000,840.0,401.092159
16,10,30,900.0,1335.166667,1403.150697,395.166667,940.0,463.150697
25,10,50,900.0,1408.400000,1394.872664,448.400000,960.0,434.872664
29,20,30,100.0,110.000000,171.488302,-40.000000,150.0,21.488302
42,30,10,800.0,1274.693548,1270.235637,434.693548,840.0,430.235637
43,30,10,900.0,1564.387097,1463.147829,624.387097,940.0,523.147829
44,30,10,1000.0,2181.666667,1762.987588,1141.666667,1040.0,722.987588


In [44]:
sum(result.loc[vip_num]['Total_Profit'])

17034.46729140899